In [ ]:
'''
    Importing the data from the ROOT files
    All root data files can be obtained through the open data 
    program from the ATLAS collaboration. 
    They can be found here:
        http://opendata.atlas.cern/samples-13tev/
    All data files are in root format and require 
    the installation of ROOT's python interface. 
    ROOT is a CERN C++ Library which can be found here:
        https://root.cern.ch/
'''

import numpy as np
import ROOT
from ROOT import TMath
from DataLoader import LoadDataAndProcess

ttH = ROOT.TFile.Open("mc_341081.ttH125_gamgam.GamGam.root")
ggH = ROOT.TFile.Open("mc_343981.ggH125_gamgam.GamGam.root")
WWH = ROOT.TFile.Open("mc_345041.VBFH125_gamgam.GamGam.root")
data1 = ROOT.TFile.Open("data_A.GamGam.root")
data2 = ROOT.TFile.Open("data_B.GamGam.root")
data3 = ROOT.TFile.Open("data_C.GamGam.root")
data4 = ROOT.TFile.Open("data_D.GamGam.root")

# Channel Dictionary
Channels = {"ttH":ttH.Get("mini"),
            "ggH":ggH.Get("mini"),
            "WWH":WWH.Get("mini"), 
            'data1':data1.Get('mini'), 
            'data2':data2.Get('mini'),
            'data3':data3.Get('mini'),
            'data4':data4.Get('mini')
        }

# Loading the data into the dicrtionaries
for channel in Channels:
    print("Channel:",channel,"has",Channels[channel].GetEntries(),"entries")

# This outputs refer to the targets for the Neural Network
# Initially all signal channels are differentiated from each other
OutputMap = {'ttH':[1,0,0,0],'ggH':[0,1,0,0],'WWH':[0,0,1,0], 
             'data1':[0,0,0,1], 'data2':[0,0,0,1], 'data3':[0,0,0,1], 'data4':[0,0,0,1]}


# Sort Photons by Energy
def SortAndFlatten(Particles, SortFunction):
    datapoint = []
    Particles.sort(key = SortFunction)
    for particle in Particles:
        datapoint.append(particle.E())
        datapoint.append(particle.Px())
        datapoint.append(particle.Py())
        datapoint.append(particle.Pz())
    return datapoint

# Compute invariant mass and sort by energy
def SortFlattenAndInvariantMass(Particles, SortFunction):
    datapoint = []
    Particles.sort(key = SortFunction)
    datapoint.append(np.sqrt(2*(Particles[0].E()**2-Particles[0].Px()**2-Particles[0].Py()**2-Particles[0].Pz()**2)))
    for particle in Particles:
        datapoint.append(particle.Px())
        datapoint.append(particle.Py())
        datapoint.append(particle.Pz())
    return datapoint

'''
    Loading both datasets from the ROOT files into 
    we also apply a sorting to both datasets

'''

UnprocessedLambda  = lambda Photons : SortAndFlatten( Photons, lambda Photon : -Photon.E() )
UnprocessedDataset = LoadDataAndProcess(Channels, OutputMap, UnprocessedLambda, 50000 )

PreprocessedLambda  = lambda Photons : SortFlattenAndInvariantMass(Photons, lambda Photon : -Photon.E() )
PreprocessedDataset = LoadDataAndProcess(Channels, OutputMap, PreprocessedLambda, 50000)

In [ ]:
from Model import Model

UnProcessedModel  = Model("Unprocessed Model" ,"./Unprocessed" ,UnprocessedDataset)
PreprocessedModel = Model("Preprocessed Model","./Preprocessed",PreprocessedDataset)

In [ ]:
UnProcessedModel.train(0.35,100,True)
PreprocessedModel.train(0.35,100,True)

In [ ]:
UnProcessedModel.analyze()
PreprocessedModel.analyze()